In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import tensorflow as tf
import os
import numpy as np
import tensorflow_io as tfio

from helper_functions import (
    convert_prefetchdataset_to_numpy_arrays,
    lite_model_from_file_predicts_dataset,
    get_file_size, 
    convert_bytes,
    evaluate_prediction,
    full_int_model_predict
)


from sklearn.metrics import confusion_matrix

2024-07-28 22:10:55.849041: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-28 22:10:56.022711: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-28 22:10:56.022777: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-28 22:10:56.054550: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-28 22:10:56.090203: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-28 22:10:56.091216: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [3]:
desired_length_of_audio = 48000
sample_rate = 16000

In [6]:
# Take all audio from testing dataset and create spectrograms from them
# We will use spectrograms for models testing
directory = '../dataset/testing'

x_data = []
y_data = []
for root, dirs, files in os.walk(directory):
    for file in files:
        full_file_name = os.path.join(root, file)

        if "non_target" in str(full_file_name):
            class_encoded = 0
        elif "target" in str(full_file_name):
            class_encoded = 1

        audio, sr = tf.audio.decode_wav(tf.io.read_file(full_file_name))
        audio = tf.squeeze(audio, axis=-1)
        resampled_audio = tfio.audio.resample(audio, rate_in=48000, rate_out=sample_rate)
        audio_length = tf.shape(resampled_audio)[0]
        if audio_length < desired_length_of_audio:
            resampled_audio = tf.pad(resampled_audio, [[0, desired_length_of_audio - audio_length]], mode='CONSTANT')
        else:
            resampled_audio = resampled_audio[:desired_length_of_audio]
        resampled_audio = tf.expand_dims(resampled_audio, axis=-1).numpy()

        x_data.append(resampled_audio)
        y_data.append(class_encoded)

# input data should be in numpy array, not in list
x_data_np = np.array(x_data)
y_data_np = np.array(y_data)

2024-07-28 22:11:10.556267: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-28 22:11:10.556783: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2256] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2024-07-28 22:11:10.775379: I tensorflow_io/core/kernels/cpu_check.cc:128] Your CPU supports instructions that this TensorFlow IO binary was not compiled to use: SSE3 SSE4.1 SSE4.2 AVX AVX2 FMA
2024-07-28 22:11:10.837470: W tensorflow_io/core/kernels/audio_video_mp3_kernels.cc:271] libmp3lame.so.0 or lame functions are not available


In [7]:
for i in x_data_np:
    print(i.shape)
    break

(48000, 1)


### CNN

##### CNN initial model in keras format

In [10]:
cnn_initial_model_path = '../time_series_models_from_notebooks/cnn/hpo/cnn_time_ser_3_conv_layer_model.keras'
convert_bytes(get_file_size(cnn_initial_model_path), "KB")

File size: 152.181 Kilobytes


In [11]:
cnn_initial_model = tf.keras.models.load_model(cnn_initial_model_path)
# input data should be in numpy array
y_pred_prob = cnn_initial_model.predict(x_data_np, verbose=0)
y_pred = tf.argmax(y_pred_prob, axis=1).numpy()

# Evaluate
evaluate_prediction(y_data_np, y_pred)

Accuracy: 98.56%
Recall: 96.51%
Precision: 99.10%
F1-score: 97.79%


##### CNN tf Lite model (without any additional quantization techniques)

In [13]:
cnn_tflite_model_path = '../time_series_models_from_notebooks/cnn/quantized/cnn_time_series.tflite'
convert_bytes(get_file_size(cnn_tflite_model_path), "KB")

y_pred = lite_model_from_file_predicts_dataset(cnn_tflite_model_path, x_data_np, y_data_np)

File size: 44.035 Kilobytes


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


Accuracy: 98.56%
Recall: 96.51%
Precision: 99.10%
F1-score: 97.79%


##### CNN tf Lite model + Post Training Dynamic range quantization

In [14]:
cnn_drq_model_path = '../time_series_models_from_notebooks/cnn/quantized/cnn_time_series_drq.tflite'
convert_bytes(get_file_size(cnn_drq_model_path), "KB")

y_pred = lite_model_from_file_predicts_dataset(cnn_drq_model_path, x_data_np, y_data_np)

File size: 17.953 Kilobytes
Accuracy: 98.56%
Recall: 96.51%
Precision: 99.10%
F1-score: 97.79%


##### CNN tf Lite model + Float 16 quantization

In [17]:
cnn_float16q_model_path = '../time_series_models_from_notebooks/cnn/quantized/cnn_time_series_float16q.tflite'
convert_bytes(get_file_size(cnn_float16q_model_path), "KB")

y_pred = lite_model_from_file_predicts_dataset(cnn_float16q_model_path, x_data_np, y_data_np)

File size: 27.406 Kilobytes
Accuracy: 98.56%
Recall: 96.51%
Precision: 99.10%
F1-score: 97.79%


##### CNN Float Fallback

In [18]:
cnn_float_fallback_model_path = '../time_series_models_from_notebooks/cnn/quantized/cnn_time_series_float_fallback_q.tflite'
convert_bytes(get_file_size(cnn_float_fallback_model_path), "KB")

y_pred = lite_model_from_file_predicts_dataset(cnn_float_fallback_model_path, x_data_np, y_data_np)

File size: 18.883 Kilobytes
Accuracy: 98.42%
Recall: 96.29%
Precision: 98.88%
F1-score: 97.57%


##### CNN tf Lite model + Full integer quantization

In [19]:
cnn_full_int_q_model_path = '../time_series_models_from_notebooks/cnn/quantized/cnn_time_series_full_int_q.tflite'
convert_bytes(get_file_size(cnn_full_int_q_model_path), "KB")

y_pred = full_int_model_predict(cnn_full_int_q_model_path, x_data_np)
evaluate_prediction(y_data_np, y_pred)

File size: 18.906 Kilobytes
Accuracy: 87.51%
Recall: 62.23%
Precision: 99.65%
F1-score: 76.61%


### SqueezeNet

##### SqueezeNet initial model in keras format

In [25]:
squeezenet_initial_model_path = '../time_series_models_from_notebooks/squeezenet/squeezenet30%_time_series_16kHz_baseline.keras'
convert_bytes(get_file_size(squeezenet_initial_model_path), "KB")

File size: 531.003 Kilobytes


In [26]:
squeezenet_initial_model = tf.keras.models.load_model(squeezenet_initial_model_path)
# input data should be in numpy array
y_pred_prob = squeezenet_initial_model.predict(x_data_np, verbose=0)
y_pred = tf.argmax(y_pred_prob, axis=1).numpy()

# Evaluate
evaluate_prediction(y_data_np, y_pred)

Accuracy: 96.63%
Recall: 94.10%
Precision: 95.57%
F1-score: 94.83%


##### SqueezeNet tf Lite model (without any additional quantization techniques)

In [27]:
squeezenet_tflite_model_path = '../time_series_models_from_notebooks/squeezenet/squeezenet30%_time_series_16kHz.tflite'
convert_bytes(get_file_size(squeezenet_tflite_model_path), "MB")
y_pred = lite_model_from_file_predicts_dataset(squeezenet_tflite_model_path, x_data_np, y_data_np)

File size: 0.147 Megabytes
Accuracy: 96.63%
Recall: 94.10%
Precision: 95.57%
F1-score: 94.83%


##### SqueezeNet tf Lite model + Post Training Dynamic range quantization

In [28]:
squeezenet_tflite_drq_model_path = '../time_series_models_from_notebooks/squeezenet/squeezenet30%_time_series_16kHz_drq.tflite'
convert_bytes(get_file_size(squeezenet_tflite_drq_model_path), "KB")
y_pred = lite_model_from_file_predicts_dataset(squeezenet_tflite_drq_model_path, x_data_np, y_data_np)

File size: 89.656 Kilobytes
Accuracy: 96.63%
Recall: 94.10%
Precision: 95.57%
F1-score: 94.83%


##### SqueezeNet tf Lite model + Float 16 quantization

In [29]:
squeezenet_float16q_model_path = '../time_series_models_from_notebooks/squeezenet/squeezenet30%_time_series_16kHz_float16q.tflite'
convert_bytes(get_file_size(squeezenet_float16q_model_path), "KB")

y_pred = lite_model_from_file_predicts_dataset(squeezenet_float16q_model_path, x_data_np, y_data_np)

File size: 97.117 Kilobytes
Accuracy: 96.63%
Recall: 94.10%
Precision: 95.57%
F1-score: 94.83%


##### SqueezeNet tf Lite model + Full integer quantization

In [30]:
squeezenet_full_int_q_model_path = '../time_series_models_from_notebooks/squeezenet/squeezenet30%_time_series_16kHz_full_int_q.tflite'
convert_bytes(get_file_size(squeezenet_full_int_q_model_path), "KB")

y_pred = full_int_model_predict(squeezenet_full_int_q_model_path, x_data_np)
evaluate_prediction(y_data_np, y_pred)

File size: 87.039 Kilobytes
Accuracy: 70.78%
Recall: 68.12%
Precision: 54.45%
F1-score: 60.52%


### BNN

##### BNN initial model in keras format

In [34]:
bnn_initial_model_path = '../time_series_models_from_notebooks/bnn/hpo/bnn_time_ser_3_conv_layer_model_binary_weights.keras'
convert_bytes(get_file_size(bnn_initial_model_path), "KB")

File size: 179.52 Kilobytes


In [35]:
from larq.layers import QuantConv1D

In [36]:
bnn_initial_model = tf.keras.models.load_model(bnn_initial_model_path)
# input data should be in numpy array
y_pred_prob = bnn_initial_model.predict(x_data_np, verbose=0)
y_pred = tf.argmax(y_pred_prob, axis=1).numpy()

# Evaluate
evaluate_prediction(y_data_np, y_pred)

Accuracy: 84.28%
Recall: 84.28%
Precision: 72.42%
F1-score: 77.90%


##### BNN larq tflite

I dont know how to read this model!!!

This model gave F1-score of 92.04%

In [38]:
bnn_larq_tflite_model_path = '../time_series_models_from_notebooks/bnn/hpo/bnn_time_ser_lq.tflite'
convert_bytes(get_file_size(bnn_larq_tflite_model_path), "KB")

# y_pred = full_int_model_predict(bnn_larq_tflite_model_path, x_data_np)
# evaluate_prediction(y_data_np, y_pred)

File size: 41.156 Kilobytes


##### BNN tf tflite

In [39]:
bnn_tf_tflite_model_path = '../time_series_models_from_notebooks/bnn/hpo/bnn_time_ser_tf_lite.tflite'
convert_bytes(get_file_size(bnn_tf_tflite_model_path), "KB")

y_pred = full_int_model_predict(bnn_tf_tflite_model_path, x_data_np)
evaluate_prediction(y_data_np, y_pred)

File size: 44.512 Kilobytes
Accuracy: 84.28%
Recall: 84.28%
Precision: 72.42%
F1-score: 77.90%


##### BNN full int quant

In [40]:
bnn_full_int_model_path = '../time_series_models_from_notebooks/bnn/hpo/bnn_time_ser_full_int_q.tflite'
convert_bytes(get_file_size(bnn_full_int_model_path), "KB")

y_pred = full_int_model_predict(bnn_full_int_model_path, x_data_np)
evaluate_prediction(y_data_np, y_pred)

File size: 25.172 Kilobytes
Accuracy: 64.54%
Recall: 98.47%
Precision: 48.08%
F1-score: 64.61%


### Time Transformer

In [47]:
transformer_initial_model_path = '../time_series_models_from_notebooks/tiny_transformer/hpo/tiny_transformer_time_ser_final.keras'
convert_bytes(get_file_size(transformer_initial_model_path), "KB")

File size: 162.422 Kilobytes


In [45]:
transformer_initial_model = tf.keras.models.load_model(transformer_initial_model_path)
# input data should be in numpy array
y_pred_prob = transformer_initial_model.predict(x_data_np, verbose=0)
y_pred = tf.argmax(y_pred_prob, axis=1).numpy()

# Evaluate
evaluate_prediction(y_data_np, y_pred)

Accuracy: 96.27%
Recall: 94.98%
Precision: 93.75%
F1-score: 94.36%


##### Transformer tflite

In [48]:
transformer_tflite_model_path = '../time_series_models_from_notebooks/tiny_transformer/quantized/tiny_transformer_time_series.tflite'
convert_bytes(get_file_size(transformer_tflite_model_path), "KB")

y_pred = full_int_model_predict(transformer_tflite_model_path, x_data_np)
evaluate_prediction(y_data_np, y_pred)

File size: 43.238 Kilobytes
Accuracy: 96.27%
Recall: 94.98%
Precision: 93.75%
F1-score: 94.36%


##### Transformer Dynamic range quantization

In [49]:
transformer_drq_model_path = '../time_series_models_from_notebooks/tiny_transformer/quantized/tiny_transformer_time_series_drq.tflite'
convert_bytes(get_file_size(transformer_drq_model_path), "KB")

y_pred = full_int_model_predict(transformer_drq_model_path, x_data_np)
evaluate_prediction(y_data_np, y_pred)

File size: 26.188 Kilobytes
Accuracy: 96.27%
Recall: 94.98%
Precision: 93.75%
F1-score: 94.36%


##### Transformer Float16

In [50]:
transformer_float16_model_path = '../time_series_models_from_notebooks/tiny_transformer/quantized/tiny_transformer_time_series_float16q.tflite'
convert_bytes(get_file_size(transformer_float16_model_path), "KB")

y_pred = full_int_model_predict(transformer_float16_model_path, x_data_np)
evaluate_prediction(y_data_np, y_pred)

File size: 33.258 Kilobytes
Accuracy: 96.27%
Recall: 94.98%
Precision: 93.75%
F1-score: 94.36%


##### Transformer Float Fallback

In [51]:
transformer_float_fallback_model_path = '../time_series_models_from_notebooks/tiny_transformer/quantized/tiny_transformer_time_series_float_fallback_q.tflite'
convert_bytes(get_file_size(transformer_float_fallback_model_path), "KB")

y_pred = full_int_model_predict(transformer_float_fallback_model_path, x_data_np)
evaluate_prediction(y_data_np, y_pred)

File size: 29.336 Kilobytes
Accuracy: 82.91%
Recall: 48.91%
Precision: 98.25%
F1-score: 65.31%


##### Transformer Full int quant

In [53]:
transformer_full_int_model_path = '../time_series_models_from_notebooks/tiny_transformer/quantized/tiny_transformer_time_series_full_int_q.tflite'
convert_bytes(get_file_size(transformer_full_int_model_path), "KB")

y_pred = full_int_model_predict(transformer_full_int_model_path, x_data_np)
evaluate_prediction(y_data_np, y_pred)

File size: 29.352 Kilobytes
Accuracy: 71.86%
Recall: 14.63%
Precision: 98.53%
F1-score: 25.48%
